####  CNN based on ResNet50 ####

In [1]:
import sys, json, os

In [2]:
import pandas as pd
import numpy as np

In [3]:
import skimage.io

In [29]:
#from tensorflow.python.keras.applications import ResNet50
import tensorflow
from tensorflow import keras
tensorflow.__version__

'1.15.0'

In [5]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np


In [11]:
from tensorflow.keras.models import model_from_json, save_model, load_model

In [4]:
#from tensorflow.python.keras.applications import EfficientNetB3

In [8]:
import tensorflow_hub as hub

In [17]:
newmodel = hub.Module('b4')

In [21]:
embed = hub.KerasLayer('b4')

In [33]:
model = tensorflow.keras.Sequential()

In [34]:
model.add(embed)

In [35]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [8]:
from panda_bvv_config import *


In [9]:
model_name = os.path.join(note_path, 'model_panda.h5')
checkpoint_name = os.path.join(note_path, 'model_panda_check')

In [10]:
bsize = 32
num_epochs = 20

In [11]:
temp_arr = []
for path, subdirs, files in os.walk(train_cnn):
    for name in files:
        temp_arr.append(name)
num_train_files = len(temp_arr)
temp_arr = []
for path, subdirs, files in os.walk(valid_cnn):
    for name in files:
        temp_arr.append(name)
num_valid_files = len(temp_arr)

In [12]:
image_sizey, image_sizex, _ = skimage.io.imread(os.path.join(train_cnn,'isup0',\
                                                os.listdir(train_cnn+'/isup0')[0])).shape



In [13]:
s_per_epoch= num_train_files//bsize
val_steps = num_valid_files//bsize

In [14]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [15]:
train_generator = data_generator.flow_from_directory(
        train_cnn,
        target_size=(image_sizey, image_sizex),
        batch_size= bsize,
        class_mode='categorical')



Found 7486 images belonging to 6 classes.


In [16]:
validation_generator = data_generator.flow_from_directory(
        valid_cnn,
        target_size=(image_sizey, image_sizex),
        batch_size = bsize,
        class_mode='categorical')


Found 1894 images belonging to 6 classes.


In [18]:
callbacks_list = [
        keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor='val_loss',
        mode = 'min',
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=5,
        verbose=1,
        restore_best_weights = True
        ),
        
        keras.callbacks.ModelCheckpoint(
        filepath= checkpoint_name +".{epoch:02d}.h5",
        monitor='val_loss',
        mode = 'auto',
        save_weights_only = False,
        save_freq = 'epoch',
        save_best_only=False
        ),
    
        #keras.callbacks.ReduceLROnPlateau(
        #monitor='val_loss',
        #factor=0.1,
        #patience=6,
        #)
]

In [19]:
resnet = ResNet50(include_top=False, pooling='avg', weights='imagenet')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [20]:
new_model = Sequential()
new_model.add(resnet)
new_model.add(Flatten())
new_model.add(Dense(512, activation='relu'))
new_model.add(Dense(num_classes, activation='softmax'))

In [28]:
#new_model.layers[0].summary()

In [27]:
#new_model.summary()

In [18]:
# Say not to train first layer (ResNet) model. It is already trained
new_model.layers[0].trainable = False

In [19]:
new_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
new_model.fit_generator(
        train_generator,
        steps_per_epoch=s_per_epoch,
        epochs = num_epochs,
        callbacks=callbacks_list,
        validation_data=validation_generator,
        validation_steps=val_steps)

Epoch 1/2
20/20 [==============================] - 629s 31s/step - loss: 1.8496 - acc: 0.2828 - val_loss: 1.9098 - val_acc: 0.2562
Epoch 2/2
20/20 [==============================] - 593s 30s/step - loss: 1.6811 - acc: 0.3234 - val_loss: 1.8938 - val_acc: 0.2406


In [23]:
new_model.save(os.path.join(base_path, model_name))

In [29]:
import os
module_name = 'respca_simple_fit'

os.system('jupyter nbconvert --to python ' + module_name + '.ipynb')
with open(module_name + '.py', 'r') as f:
    lines = f.readlines()
with open(module_name + '.py', 'w') as f:
    for line in lines:
        if 'nbconvert --to python' in line:
            break
        else:
            f.write(line)